## 简介

爱好四轴飞行器，还想了解其飞行的数学模型。编写一个简单的仿真程序，可能是最完整和最经济的学习手段。
1. 首先，**根据牛顿-欧拉方程建立四旋翼飞行器的动力学和运动学模型**，从而可以根据飞行器的实时状态（受力、位姿、线速度和角速度等数据），计算输出其实时位姿变化（线加速度和角加速度），并可视化仿真程序。
2. 其次，进行线性模型化简，**根据 PID 控制律设计底层飞行控制器**，以飞行器的实时状态和飞行指令（定点等）为输入，实时计算对应的螺旋桨转速数据，并在仿真程序中进行测试和验证。

<img src="https://pic2.zhimg.com/v2-464d5c4552d44b407c3a1d9b8a774084_1440w.gif" alt="仿真示意图" title="示意图来自知乎抱歉忘了原作者是谁">

参考资料
* [多旋翼飞行器设计与控制](https://study.163.com/course/courseMain.htm?courseId=1003715005)
* [玖辞丶四旋翼飞行器建模](https://zhuanlan.zhihu.com/p/349306054)

## 坐标系

对于四旋翼模型来说，需要两个坐标系。初始状态下，这个两个坐标系完全重合，且和 Three.js 和 Blender 的坐标系定义一致。
* 全局坐标系/大地坐标系/惯性坐标系，XYZ 的正方向分别对应北东地方向 (NED)
* 机体坐标系，XYZ 的正方向分别对应前右下方向

<img src="frame.png" />

螺旋桨位置：
* $ \omega_1 = (1, -1, 0) $ 机体的左前方
* $ \omega_2 = (1, 1, 0) $ 机体的右前方
* $ \omega_3 = (-1, 1, 0) $ 机体的右后方
* $ \omega_4 = (-1, -1, 0) $ 机体的左后方

$ \phi \ \theta \ \psi $ 分别表示绕 $ X $ 轴旋转的**翻滚角 Roll**、绕 $ Y $ 轴旋转的**俯仰角 Pitch** 以及绕 $ Z $ 轴旋转的**偏航角 Yaw**，即欧拉角。旋转方向由右手螺旋定义。

<img src="https://pic1.zhimg.com/80/v2-f3101ad00adbc902f12d9ffcedae4e24_720w.jpg" width="30%" />



任一机体坐标系下的向量 $ p^b $ 左乘旋转矩阵 $ R_b^e $ 即可得到地球坐标系下对应的向量 $ p^e $

$ p^e = R_b^e p^b= 
\begin{bmatrix} 
cos(\theta)cos(\psi) & cos(\psi)sin(\theta)sin(\phi)-sin(\psi)cos(\phi) & cos(\psi)sin(\theta)cos(\phi)+sin(\psi)sin(\phi) \\ 
cos(\theta)sin(\psi) & sin(\psi)sin(\theta)sin(\phi)+cos(\psi)cos(\phi) & sin(\psi)sin(\theta)cos(\phi)-cos(\psi)sin(\phi) 
\\ -sin(\theta) & sin(\phi)cos(\theta) & cos(\phi)cos(\theta)
\end{bmatrix} p^b $


## 控制效率模型

（第06讲_动态模型和参数测量V2.pdf 第17页）控制效率模型的作用是将**螺旋桨旋转所产生的拉力和反扭力矩**计算出来。

$ f = c_T (\omega_1^2+\omega_2^2+\omega_3^2+\omega_4^2)$
* **未知数** $ f $ 是螺旋桨拉力（机体轴）
* $ c_T $ 是螺旋桨拉力系数，是通过试验测量获得的常数数值
* $ \omega_i $ 是第 i 个螺旋桨（电机）转速

$
\begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \end{bmatrix} = 
\begin{bmatrix} 
d c_T (\frac{\sqrt{2}}{2} \omega_1^2 - \frac{\sqrt{2}}{2} \omega_2^2 - \frac{\sqrt{2}}{2} \omega_3^2 + \frac{\sqrt{2}}{2} \omega_4^2) \\
d c_T (\frac{\sqrt{2}}{2} \omega_1^2 + \frac{\sqrt{2}}{2} \omega_2^2 - \frac{\sqrt{2}}{2} \omega_3^2 - \frac{\sqrt{2}}{2} \omega_4^2) \\
c_M (\omega_1^2 - \omega_2^2 + \omega_3^2 - \omega_4^2) 
\end{bmatrix}
$

* **未知数** $ \tau $ 表示螺旋桨在机体轴上产生的力矩三维向量 $ \tau = \begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \end{bmatrix} $
* $ d $ 表示机体中心和任一电机的距离(m)
* $ c_M $ 表示螺旋桨力矩系数，是通过试验测量获得的常数数值


全权, 戴训华, 魏子博, 等. 一种测量小型飞行器**转动惯量与惯性积**的方法
* c_T = 1.105e-05; 螺旋桨拉力系数
* c_M = 1.779e-07*2; 螺旋桨力矩系数
* I_xx = 0.0211; 四旋翼x轴转动惯量(kg·m^2)
* I_yy = 0.0219; 四旋翼y轴转动惯量(kg·m^2)
* I_zz = 0.0366; 四旋翼z轴转动惯量(kg·m^2)
* J_RP = 0.0001287; 整个电机转子和螺旋桨绕转轴的总转动惯量(kg·m^2)

## 位置动力学模型

位置动力学模型 (Position Dynamic Model)，即质心位置的平动是根据牛顿第二定律 $ F = ma $ 进行描述：

$
\begin{equation}
m \dot v^e = G^e + f^b \tag{1}
\end{equation}
$
* **未知数** $ \dot v^e $ 是地球坐标系下沿 $ x \ y \ z $ 轴的**线速度的导数**三维向量，即加速度 $ \dot v^e = \begin{bmatrix} \dot v^e_x \\ \dot v^e_y \\ \dot v^e_z \end{bmatrix} $
* $ m $ 是飞行器的总质量（千克）
* $ G^e $ 是地球坐标系下飞行器的重力三维向量，即重力的大小乘以方向 $ G^e = G \times c^e $ 。可以简单认为地球是平的，所以重力方向始终是地球坐标系 Z+ 方向，也可以动态计算。
* $ f^b $ 是机体坐标系下的合拉力三维向量，即拉力的大小乘以方向 $ f^b = f \times \begin{bmatrix} 0 \\ 0 \\ -1 \end{bmatrix} $，拉力大小可以根据螺旋桨实时转速和拉力系数计算得到。

需要将机体坐标系下的合拉力的向量变换到地球坐标系下统一表达：

$
m \dot v^e = G \times c^e + f \times R_b^e \begin{bmatrix} 0 \\ 0 \\ -1 \end{bmatrix} \tag{2}
$

* $ R_b^e $ 是从机体坐标系到地球坐标系的变换矩阵，根据机身姿态计算得到。


等式两边同时除以质量 $ m $ 得到：

$
\begin{equation}
\dot v^e = g \times c^e + \frac {f} {m} \times R_b^e \begin{bmatrix} 0 \\ 0 \\ -1 \end{bmatrix} \tag{3}
\end{equation}
$
* $ g $ 是物理常量重力加速度 $ g = G \div m = 9.8 m/s^2 $

## 姿态动力学模型

绕质心的转动由欧拉方程 $ M = J \dot \omega + \omega \times J \omega $ 描述：

$
\begin{equation}
J \dot \omega^b + \omega^b \times J \omega^b = G_a + \tau \tag{1}
\end{equation}
$

* **未知数** $ \dot \omega^b $ 表示在机体坐标系下的角加速度三维向量
* $ \omega^b $ 表示在机体坐标系下的角速度三维向量
* $ J $ 为3×3的惯性矩阵
* $ G_a $ 表示陀螺力矩
* $ \tau $ 表示螺旋桨在机体轴上产生的力矩三维向量 $ \tau = \begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \end{bmatrix} $

## 其他

ISSUE:
* Python 并行处理多用户（各自的模型）的计算能力有限，可能 C/GO 更合适

~~~mermaid
gantt
    title 每一瞬间可以是任意长度的固定时间片段
    dateFormat s:SSS
    axisFormat %S.%L
    todayMarker off

    第一瞬间:done, moment_1, 0:000, 1s
    时间_1:active, time_1, 0:000, 0:200
    计算_1:crit, active, calc_1, 0:200, 0:800
    
    第二瞬间:done, moment_2, 1:000, 1s
    时间_2:active, time_2, 0:200, 1:200
    计算_2:crit, active, calc_2, 1:200, 1:800
    
    第三瞬间:done, moment_3, 2:000, 1s
    时间_3:active, time_3, 1:200, 2:200
    计算_3:crit, active, calc_3, 2:200, 2:800
~~~